# Simple replication of "Thread of Thought Unraveling Chaotic Contexts"
In the paper the authors present results from experiments using various datasets, including PopQA, EntityQ, and a custom Multi-Turn Conversation Response (MTCR) dataset. They report performance improvements compared to other prompting techniques. However, the paper does not provide sufficient information for full study replication -- such as which dataset was used for retrieval to create "chaotic context" or where to access the custom Multi-Turn Conversation Response (MTCR) dataset and therefore I could not independently verify the results.

Instead, in collaboration with Claude Sonnet 3.5 I created two "chaotic context" prompts:
1. A prompt about Roman history
2. A prompt from a 3 July 2024 news article about Japanese regulation concerning the phase out of floppy disks (https://arstechnica.com/gadgets/2024/07/japans-government-finally-exits-90s-ends-floppy-disk-use/). This ensures the model could not have any prior information about the correct answer.

## Prompt 1: Roman history
The model may have memorized some aspects of Roman history, which may help it with this factual version of a "chaotic context" prompt.

### Set up context and functions

In [9]:
import ollama

# This context was generated in collaboration with Claude Sonnet 3.5. The correct answer is Odoacer.
# This is a multi-hop question, which involves connecting the following pieces of information to arrive at the correct answer:
# 1. Identify Romulus Augustulus as the last Western Roman Emperor (combining information from points 3 and 6).
# 2. Determine that Ravenna was where he ruled (from point 9) and that Ravenna is in Italy (from point 1).
# 3. Identify Odoacer as the leader who took control of Italy in 476 CE, the year the last emperor was deposed (from point 7).

chaotic_context = """Based on the following information, who was the leader of the group that conquered the city where the last Western Roman Emperor ruled?

- The city of Ravenna, located in Italy, became the capital of the Western Roman Empire in 402 CE.
- Julius Caesar crossed the Rubicon river in 49 BCE, sparking a civil war.
- Romulus Augustulus began his reign as Western Roman Emperor in 475 CE.
- The Ostrogothic king Theoderic the Great defeated Odoacer in 493 CE.
- Constantinople fell to the Ottoman Turks in 1453 CE, ending the Byzantine Empire.
- The last Western Roman Emperor was deposed in 476 CE.
- Odoacer, a Germanic chieftain, took control of Italy in 476 CE.
- The Visigoths sacked Rome in 410 CE under the leadership of Alaric I.
- Romulus Augustulus ruled from the city of Ravenna until his deposition.
- The Vandals captured the city of Carthage in 439 CE."""

cot_prompt = "Let's think step by step."
thot_prompt = "Walk me through this context in manageable parts step by step, summarizing and analyzing as we go."

def question_answerer(chaotic_context, prompt):
    model = 'llama3'
    messages = [
        {
            'role': 'user',
            'content': chaotic_context + "\n\n" + prompt
        }
    ]

    # Make the request to the Ollama API
    response = ollama.chat(model=model, messages=messages, stream=False)
    return response["message"]["content"]

"Let's break it down step by step!\n\nWe know that the last Western Roman Emperor was deposed in 476 CE, and Romulus Augustulus began his reign as Western Roman Emperor in 475 CE. This means that Romulus Augustulus was still ruling when he was eventually deposed.\n\nSince Romulus Augustulus ruled from Ravenna until his deposition, we can conclude that Ravenna was the city where the last Western Roman Emperor ruled.\n\nNow, to find out who led the group that conquered this city, let's look at the timeline. We know Odoacer took control of Italy in 476 CE, and Romulus Augustulus was deposed around the same time. This suggests that Odoacer might be the leader we're looking for!\n\nAm I correct?"

### Vanilla prompting without chain of thought or thread of thought (Roman history)

In [22]:
for i in range(1, 11):
    print(f"Response {i}:")
    print(question_answerer(chaotic_context, "")) # Print results and manually save file
    print("\n\n")
    print("---------")

Response 1:
According to the information, Odoacer was the leader of the group that conquered the city where the last Western Roman Emperor ruled. He took control of Italy in 476 CE and deposed the last Western Roman Emperor.



---------
Response 2:
According to the information, Odoacer was the leader of the group that conquered the city where the last Western Roman Emperor ruled. Specifically, he took control of Italy and deposed the last Western Roman Emperor in 476 CE.



---------
Response 3:
According to the information, Odoacer was the leader of the group that conquered the city where the last Western Roman Emperor ruled. Specifically, he took control of Italy and deposed the last Western Roman Emperor in 476 CE.



---------
Response 4:
According to the information, Odoacer, a Germanic chieftain, took control of Italy and deposed the last Western Roman Emperor in 476 CE. Therefore, Odoacer was the leader of the group that conquered the city where the last Western Roman Emperor r

### Standard chain of thought prompting (Roman history)

In [20]:
for i in range(1, 11):
    print(f"Response {i}:")
    print(question_answerer(chaotic_context, cot_prompt)) # Print results and manually save file
    print("\n\n")
    print("---------")

Response 1:
A nice historical puzzle! Let's break it down:

1. The last Western Roman Emperor was deposed in 476 CE.
2. Romulus Augustulus began his reign as Western Roman Emperor in 475 CE, and ruled from the city of Ravenna until his deposition (in 476 CE).
3. Odoacer, a Germanic chieftain, took control of Italy in 476 CE, which means he was the one who deposed Romulus Augustulus.
4. Therefore, Odoacer was the leader of the group that conquered the city where the last Western Roman Emperor ruled (Ravenna).

So, the answer is: Odoacer!



---------
Response 2:
Let's work backwards to find out who was the leader of the group that conquered the city where the last Western Roman Emperor ruled.

We know that Romulus Augustulus, the last Western Roman Emperor, was deposed in 476 CE. We also know that he ruled from Ravenna, which became the capital of the Western Roman Empire in 402 CE.

In 476 CE, Odoacer, a Germanic chieftain, took control of Italy. So, it's likely that Odoacer is the lea

### Thread of thought prompting (Roman history)

In [21]:
for i in range(1, 11):
    print(f"Response {i}:")
    print(question_answerer(chaotic_context, thot_prompt)) # Print results and manually save file
    print("\n\n")
    print("---------")


Response 1:
Let's break down the information into smaller chunks and analyze each part step by step.

**Part 1: Early History**

First, let's look at Julius Caesar crossing the Rubicon river in 49 BCE, which sparked a civil war. This event is significant because it marks the beginning of Caesar's rise to power and sets the stage for the Roman Empire.

**Summary:** Julius Caesar crossed the Rubicon river in 49 BCE, starting a civil war that would lead to his eventual rise to power.

**Part 2: Western Roman Empire**

Next, we learn that Ravenna became the capital of the Western Roman Empire in 402 CE. This is important because it tells us when the Western Roman Empire was established and where its capital was located.

**Summary:** The city of Ravenna became the capital of the Western Roman Empire in 402 CE.

**Part 3: Rise and Fall of Western Roman Emperor**

Now, let's examine Romulus Augustulus' reign as Western Roman Emperor. He began his rule in 475 CE, but was eventually deposed. W

## Prompt 2: Japanese floppy disks
The model may have memorized some aspects of Roman history, which may help it with the prior version of a "chaotic context" prompt. This second version leverages facts from a 3 July 2024 news article about Japanese regulation concerning the phase out of floppy disks (https://arstechnica.com/gadgets/2024/07/japans-government-finally-exits-90s-ends-floppy-disk-use/). This ensures the model could not have any prior information about the correct answer.

In [24]:
import ollama

# This context was generated in collaboration with Claude Sonnet 3.5. The correct answer is Odoacer.
# This is a multi-hop question, which involves connecting the following pieces of information to arrive at the correct answer:
# 1. Identify that the complete elimination of floppy disks occurred on June 28, 2024 (point 4).
# 2. Recognize that the government stopped requiring physical media two years before this complete elimination (point 5).
# 3. Calculate that two years before 2024 is 2022.

chaotic_context = """Based on the following information, in what year did the Japanese government stop requiring physical media for business filings and citizen submission forms?

- Japan's Digital Agency was established during the COVID-19 pandemic to modernize government technology.
- Digital Minister Taro Kono declared a "war on floppy discs" about two years ago.
- The US Air Force used 8-inch floppies until 2019.
- As of June 28, 2024, Japan's government eliminated the use of floppy disks in all its systems, except for one environmental regulation.
- Two years before the complete elimination of floppy disks, the government stopped requiring physical media for 1,900 types of submissions.
- The Institute for Management Development's 2023 World Digital Competitiveness Ranking listed Japan as number 32 out of 64 economies.
- Many workplaces in Japan still opt for fax machines over emails.
- Sony, the last company to manufacture floppy disks, stopped production in 2011.
- Floppy disks typically had a storage capacity that maxed out at 1.44MB, with some versions reaching 2.88MB.
- Chuck E. Cheese was reported to be using floppy disks for its animatronics as recently as January 2023."""


cot_prompt = "Let's think step by step."
thot_prompt = "Walk me through this context in manageable parts step by step, summarizing and analyzing as we go."

def question_answerer(chaotic_context, prompt):
    model = 'llama3'
    messages = [
        {
            'role': 'user',
            'content': chaotic_context + "\n\n" + prompt
        }
    ]

    # Make the request to the Ollama API
    response = ollama.chat(model=model, messages=messages, stream=False)
    return response["message"]["content"]

### Vanilla prompting without chain of thought or thread of thought (Japanese floppy disks)

In [25]:
for i in range(1, 11):
    print(f"Response {i}:")
    print(question_answerer(chaotic_context, "")) # Print results and manually save file
    print("\n\n")
    print("---------")

Response 1:
According to the information, two years before the complete elimination of floppy disks, which occurred in June 28, 2024, the government stopped requiring physical media for 1,900 types of submissions. This would have been in June 2022.



---------
Response 2:
According to the information, Japan's government stopped requiring physical media (floppy disks) for business filings and citizen submission forms two years before June 28, 2024. This means that it happened in 2022.



---------
Response 3:
According to the information, the Japanese government stopped requiring physical media (i.e., floppy disks) for business filings and citizen submission forms about two years before they completely eliminated their use in all systems. Therefore, we can calculate that this happened:

June 28, 2024 - 2 years = June 28, 2022

So, the Japanese government stopped requiring physical media (floppy disks) for business filings and citizen submission forms in 2022.



---------
Response 4:
A

### Standard chain of thought prompting (Japanese floppy disks)

In [26]:
for i in range(1, 11):
    print(f"Response {i}:")
    print(question_answerer(chaotic_context, cot_prompt)) # Print results and manually save file
    print("\n\n")
    print("---------")

Response 1:
Let's break it down!

We know that the government stopped requiring physical media for 1,900 types of submissions "two years before the complete elimination of floppy disks".

Since the government completely eliminated floppy disks in all its systems except one environmental regulation as of June 28, 2024, we can infer that they stopped requiring physical media for these 1,900 types of submissions two years prior to that.

So, if we subtract two years from June 28, 2024, we get:

June 28, 2022

That's when the government stopped requiring physical media for business filings and citizen submission forms!



---------
Response 2:
A nice puzzle! Let's break it down:

1. We know that the government stopped requiring physical media (floppy disks) for 1,900 types of submissions "two years before" they eliminated floppy disks completely.
2. This event happened on June 28, 2024, when Japan's government eliminated floppy disks in all its systems, except for one environmental regulat

### Thread of thought prompting (Japanese floppy disks)

In [27]:
for i in range(1, 11):
    print(f"Response {i}:")
    print(question_answerer(chaotic_context, thot_prompt)) # Print results and manually save file
    print("\n\n")
    print("---------")

Response 1:
Let's break down the information into manageable parts and summarize each section before moving on to the next one.

**Part 1: Background Information**

The text mentions that Japan's Digital Agency was established during the COVID-19 pandemic to modernize government technology. This suggests that the country is making efforts to digitize its systems and improve efficiency.

No specific year is mentioned, but this information sets the stage for the rest of the passage, which appears to be discussing the elimination of floppy disks in Japan.

**Summary:** Japan's Digital Agency was established during the COVID-19 pandemic to modernize government technology.

**Part 2: The "War on Floppy Discs"**

Digital Minister Taro Kono declared a "war on floppy discs" about two years ago. This implies that there was a concerted effort to phase out floppy disks, and it seems that this effort has been successful.

No specific year is mentioned for when the declaration was made, but we'll l

## Two-stage ThoT
We could stop at one-stage ThoT and manually review the results produced by the ThoT prompt, but let's continue through the second stage and have the model use the summaries to produce a final answer.

### Romain history

In [34]:
# Function to read the text file and extract responses
def read_responses(file_path):
    with open(file_path, 'r') as file:
        data = file.read()

    # Split the text into individual responses based on the delimiter "---------"
    responses = data.split("---------")
    # Clean up each response by stripping leading/trailing whitespace
    responses = [response.strip() for response in responses if response.strip()]
    return responses

responses = read_responses("llama3_thot_roman_results.txt")
for i, response in enumerate(responses, start=1):
    final_answer = question_answerer(response, "\n\n Based on the above summary, who was the leader of the group that conquered the city where the last Western Roman Emperor ruled? \n Answer:")
    print(f"Final answer for summary response {i}:")
    print(final_answer)
    print("\n\n")
    print("---------")

Final answer for summary response 1:
According to the summary, the leader of the group that conquered the city where the last Western Roman Emperor ruled was Odoacer, a Germanic chieftain.



---------
Final answer for summary response 2:
Based on the information provided, it is not mentioned which group or leader conquered the city where the last Western Roman Emperor ruled. The text only states that Romulus Augustulus began his reign as Western Roman Emperor in 476 CE and that the last Western Roman Emperor was deposed in 476 CE. It does not provide information about the leader of the group that conquered the city or the circumstances surrounding the deposition of the emperor.



---------
Final answer for summary response 3:
According to the summary, Odoacer rose to power and took control of Italy in 476 CE, following the deposition of the last Western Roman Emperor. Therefore, Odoacer is the leader who conquered the city where the last Western Roman Emperor ruled.



---------
Fina

### Japanese floppy disks

In [35]:
responses = read_responses("llama3_thot_floppy_disks_results.txt")

for i, response in enumerate(responses, start=1):
    final_answer = question_answerer(response, "\n\n Based on the above summary, in what year did the Japanese government stop requiring physical media for business filings and citizen submission forms? \n Answer:")
    print(f"Final answer for summary response {i}:")
    print(final_answer)
    print("\n\n")
    print("---------")

Final answer for summary response 1:
According to the summary, two years before June 28, 2024, the government stopped requiring physical media (i.e., floppy disks) for 1,900 types of submissions. This would mean that they stopped requiring physical media in 2022.

So, the correct answer is 2022.



---------
Final answer for summary response 2:
According to the summary, the Japanese government stopped requiring physical media for 1,900 types of submissions about two years before the complete elimination of floppy disks (presumably around June 2022). This would mean that they stopped requiring physical media in 2020.



---------
Final answer for summary response 3:
Unfortunately, there is no specific year mentioned in this part of the information. The text only mentions the establishment of a Digital Agency during the COVID-19 pandemic and the declaration of a "war on floppy discs" by Digital Minister Taro Kono, but it does not provide a timeline for when the government stopped requiri